In [4]:
import numpy as np
import pandas as pd

In [14]:
def name_compare(name1, name2):
    name1 = str(name1).lower().replace('\'', '').replace('-', '').replace('ctr', 'center')
    name2 = str(name2).lower().replace('\'', '').replace('-', '').replace('ctr', 'center')
    if name1 in name2 or name2 in name1:
        return -1
    else:
        count1 = 0
        count2 = 0
        for part1 in name1.split():
            if part1 in name2:
                count1 += 1
        for part2 in name2.split():
            if part2 in name1:
                count2 += 1
        score = (count2 + count1) / (len(name1.split()) + len(name2.split()))
        return score

    
def num_compare(num1, num2):
    if num1 == num2:
        return -1

    num1split = num1.split('-')
    num2split = num2.split('-')
    if len(num1split) > 1 and len(num2split) == 1:
        if num2 >= num1split[0] or num2 <= num1split[1]:
            return -1

    elif len(num2split) > 1 and len(num1split) == 1:
        if num1 >= num2split[0] or num1 <= num2split[1]:
            return -1

    elif len(num1split) == 1 and len(num2split) == 1:
        try:
            int(num1split[0])
            int(num2split[0])
            return abs(int(num1) - int(num2))
        except ValueError:
            return 100
    else:
        return 100
        
    
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)    


def compare_addr(addr1, addr2):
    addr1 = addr1.lower()
    addr2 = addr2.lower()
    l1 = addr1.split(' ')
    l2 = addr2.split(' ')
    if has_numbers(addr1) and has_numbers(addr2):
        score = num_compare(l1[0], l2[0])
        try:
            if score == -1 or score < 5:
                if len(l1) == 3 and len(l2) == 3:
                    if l1[1] in l2[1] or l2[1] in l1[1]:
                        if l1[2] in l2[2] or l2[2] in l1[2]:
                            return True
                else:
                    if l1[1] in l2[1] or l2[1] in l1[1]:
                        return True
        except:
            return False
                
    else:
        if len(l1) == 2 and len(l2) == 2:
            if l1[0] in l2[0] or l2[0] in l1[0]:
                if l1[1] in l2[1] or l2[1] in l1[1]:
                    return True
        else:
            if l1[0] in l2[0] or l2[0] in l1[0]:
                return True
            
    return False


def process_addr(addr):
    return addr.replace('.', '').replace(',', '').replace('Street', 'St').replace('Avenue', 'Ave').replace(
            'Terrace', 'Ter').replace('Court', 'Ct').replace('Road', 'Rd').replace('# ', '').replace(
            '#', '').replace('Ste ', '').lower()

In [15]:
def merge(ometh, nmeth):
    # Desired Columns for final dataframe
    cols = ['address', 
            'name', 
            'latitude', 
            'longitude', 
            'In Google list (Y/N)',
            'In Bing list (Y/N)',
            'In Yelp list (Y/N)'
           ]
    
    # Merge dictionary, can be converted to dataframe after merge
    md = {}
    
    for col in cols:
        md[col] = []
        
    ometh_m = set()
    nmeth_n = set()
    
    # The merge, n^2 that tracks what is seen to not repeat
    for i in range(len(ometh)):
        for j in range(len(nmeth)):
            if j not in nmeth_n:
                
                n1 = ometh.iloc[i]['name']
                n2 = nmeth.iloc[j]['BUSINESS_NAME']
                score = name_compare(n1, n2)
                
                a1 = process_addr(ometh.iloc[i]['vicinity'])
                a2 = nmeth.iloc[j]['ADDRESS_LINE']
                print(a1)
                print(a2)
                if pd.notnull(a2):
                    a2 = process_addr(a2)

                    # If it passes these, it is considered a match
                    if (score == -1 or score > .6) and compare_addr(a1, a2):
                        md['address'].append(ometh.iloc[i]['vicinity'])
                        md['name'].append(n1)
                        md['latitude'].append(ometh.iloc[i]['lat'])
                        md['longitude'].append(ometh.iloc[i]['lon']) 
                        md['In Google list (Y/N)'].append('Y')
                        md['In Bing list (Y/N)'].append('Y')

                        ometh_m.add(i)
                        nmeth_n.add(j)
                
    # For anything not found a duplicate, adds the original record to our md
    for i in range(len(ometh)):
        if i not in ometh_m:
            n1 = ometh.iloc[i]['name']
            md['address'].append(ometh.iloc[i]['vicinity'])
            md['name'].append(n1)
            md['latitude'].append(ometh.iloc[i]['lat'])
            md['longitude'].append(ometh.iloc[i]['lon']) 
            md['In Google list (Y/N)'].append('Y')
            md['In Bing list (Y/N)'].append('N')

    # Same for the other data source
    for j in range(len(nmeth)):
        if j not in nmeth_n:
            n2 = nmeth.iloc[j]['name']
            md['address'].append(ometh.iloc[i]['vicinity'])
            md['name'].append(n1)
            md['latitude'].append(ometh.iloc[i]['lat'])
            md['longitude'].append(ometh.iloc[i]['lon']) 
            md['In Google list (Y/N)'].append('N')
            md['In Bing list (Y/N)'].append('Y')
    
    # returns the seen sets for convenience of analyzing the algorithm
    return md, ometh_m, nmeth_n

In [ ]:
google = pd.read_excel('/Users/naiwenxu/Desktop/Google Results.xlsx')
bing = pd.read_excel('/Users/naiwenxu/Desktop/Bing Results.xlsx')

results = merge(google, bing)
merged = pd.DataFrame.from_dict(results[0])
merged